In [1]:
import numpy as np
import pandas as pd
import string, xlrd, nltk, unicodedata, pickle, os, re, time
import xml.etree.cElementTree as ET
from nltk import word_tokenize
from collections import defaultdict, Counter
import difflib
import matplotlib.pyplot as plot

from bs4 import BeautifulSoup

In [2]:
df = pd.DataFrame(np.random.randint(5, 10, size=10), columns=["Numbers"])
df

,Numbers
0,5
1,8
2,8
3,5
4,5
5,6
6,8
7,6
8,6
9,8


In [3]:
unique_rows = df.index.tolist()
i = 0
while i<len(unique_rows)-1:
    j = i+1
    while j<len(unique_rows):
        print ("i is", i, "and j is", j)
        print (df.Numbers[unique_rows[i]], df.Numbers[unique_rows[j]])
        if df.Numbers[i]==df.Numbers[j]:
            unique_rows = unique_rows[:j] + unique_rows[j+1:]
            print (unique_rows)
        else:
            j += 1
    i += 1
unique_rows

i is 0 and j is 1
5 8
i is 0 and j is 2
5 8
i is 0 and j is 3
5 5
[0, 1, 2, 4, 5, 6, 7, 8, 9]
i is 0 and j is 3
5 5
[0, 1, 2, 5, 6, 7, 8, 9]
i is 0 and j is 3
5 6
[0, 1, 2, 6, 7, 8, 9]
i is 0 and j is 3
5 8
[0, 1, 2, 7, 8, 9]
i is 0 and j is 3
5 6
[0, 1, 2, 8, 9]
i is 0 and j is 3
5 6
[0, 1, 2, 9]
i is 0 and j is 3
5 8
[0, 1, 2]
i is 1 and j is 2
8 8
[0, 1]


[0, 1]

In [4]:
df.reindex(unique_rows)

,Numbers
0,5
1,8


In [5]:
df.reindex(unique_rows)

,Numbers
0,5
1,8


In [6]:
df2 = df.reindex([4, 5, 8])
df2

,Numbers
4,5
5,6
8,6


In [7]:
df

,Numbers
0,5
1,8
2,8
3,5
4,5
5,6
6,8
7,6
8,6
9,8


In [8]:
l2 = [1, 2, 3, 4, 5]
l2[:2] + l2[3:]

[1, 2, 4, 5]

In [9]:
ix=['alpha', 'beta', 'gamma', 'delta', 'epsilon']
cols = ['A', 'B', 'C', 'D']
df1 = pd.DataFrame(np.arange(20).reshape(5, 4), index=ix)
df1.columns = cols
#df1.index.name = 'Indekset'
df1.rename(columns={'C': 'Dette er kolonne C'}, inplace=True)
#df1 = df1.rename(index={'gamma': 'Charlie'})
df1

,A,B,Dette er kolonne C,D
alpha,0,1,2,3
beta,4,5,6,7
gamma,8,9,10,11
delta,12,13,14,15
epsilon,16,17,18,19


In [10]:
df1.loc['beta', 'D'] = None

In [11]:
df1

,A,B,Dette er kolonne C,D
alpha,0,1,2,3
beta,4,5,6,NaN
gamma,8,9,10,11
delta,12,13,14,15
epsilon,16,17,18,19


In [12]:
df1.dropna()

,A,B,Dette er kolonne C,D
alpha,0,1,2,3
gamma,8,9,10,11
delta,12,13,14,15
epsilon,16,17,18,19


In [13]:
df1.fillna('Wait for it...')

,A,B,Dette er kolonne C,D
alpha,0,1,2,3
beta,4,5,6,Wait for it...
gamma,8,9,10,11
delta,12,13,14,15
epsilon,16,17,18,19


In [14]:
df1

,A,B,Dette er kolonne C,D
alpha,0,1,2,3
beta,4,5,6,NaN
gamma,8,9,10,11
delta,12,13,14,15
epsilon,16,17,18,19


In [15]:
df1.rename(columns={'D': 'NixNix'}, inplace=True)
df1.fillna(666)
df1['Dette er kolonne C'].map(lambda x: x*15)

alpha       30
beta        90
gamma      150
delta      210
epsilon    270
Name: Dette er kolonne C, dtype: int64

In [16]:
df1['textKolonne'] = ["text 1", "text2", "text3", "text4", "text5"]
df1

,A,B,Dette er kolonne C,NixNix,textKolonne
alpha,0,1,2,3,text 1
beta,4,5,6,NaN,text2
gamma,8,9,10,11,text3
delta,12,13,14,15,text4
epsilon,16,17,18,19,text5


In [17]:
def capitalizeString(x):
    if type(x) is str:
        return x.upper()

In [18]:
df1.ix[:'gamma', 'Dette er kolonne C':]

,Dette er kolonne C,NixNix,textKolonne
alpha,2,3,text 1
beta,6,NaN,text2
gamma,10,11,text3


In [19]:
df1.apply(capitalizeString)

A                     None
B                     None
Dette er kolonne C    None
NixNix                None
textKolonne           None
dtype: object

In [20]:
df1

,A,B,Dette er kolonne C,NixNix,textKolonne
alpha,0,1,2,3,text 1
beta,4,5,6,NaN,text2
gamma,8,9,10,11,text3
delta,12,13,14,15,text4
epsilon,16,17,18,19,text5


In [28]:
df = pd.DataFrame(np.random.rand(4,5), columns = list('abcde'))
df

,a,b,c,d,e
0,0.515417,0.827102,0.710631,0.656196,0.294283
1,0.495557,0.056325,0.430171,0.416532,0.995915
2,0.980772,0.447714,0.358015,0.370909,0.148303
3,0.402110,0.436397,0.539040,0.418609,0.282446


In [33]:
df[df['c']>0.5].ix[:, ['b', 'c']].values

array([[ 0.82710159,  0.7106307 ],
       [ 0.43639737,  0.5390404 ]])

In [26]:
df[df['c']>0.5].ix[:, ['b', 'e']].values

array([[ 0.45396998,  0.31731025],
       [ 0.02988012,  0.06849514]])

In [13]:
from sklearn.cross_validation import StratifiedShuffleSplit
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1])
sss = StratifiedShuffleSplit(y, 3, test_size=0.5, random_state=0)
sss

StratifiedShuffleSplit(labels=[0 0 1 1], n_iter=3, test_size=0.5, random_state=0)

In [10]:
for s in sss:
    print (s)

(array([1, 2]), array([3, 0]))
(array([0, 2]), array([1, 3]))
(array([0, 2]), array([3, 1]))
